# Batch Learning

# Import Libraries

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor

import numpy as np
from rich import print

import pandas as pd

## Split data

In [2]:
data = pd.read_csv("./air.csv")
targets = data.pop("T")

data.describe()

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH
count,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000,9357.000000
mean,-34.207524,1048.990061,-159.090093,1.865683,894.595276,168.616971,794.990168,58.148873,1391.479641,975.072032,39.485380,-6.837604
std,77.657170,329.832710,139.789093,41.380206,342.333252,257.433866,321.993552,126.940455,467.210125,456.938184,51.216145,38.976670
min,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000,-200.000000
25%,0.600000,921.000000,-200.000000,4.000000,711.000000,50.000000,637.000000,53.000000,1185.000000,700.000000,34.100000,0.692300
50%,1.500000,1053.000000,-200.000000,7.900000,895.000000,141.000000,794.000000,96.000000,1446.000000,942.000000,48.600000,0.976800
75%,2.600000,1221.000000,-200.000000,13.600000,1105.000000,284.000000,960.000000,133.000000,1662.000000,1255.000000,61.900000,1.296200
max,11.900000,2040.000000,1189.000000,63.700000,2214.000000,1479.000000,2683.000000,340.000000,2775.000000,2523.000000,88.700000,2.231000


In [3]:
# TODO: Test if its really necessary to do the preprocess

encoder = LabelEncoder()

data["Time"] = encoder.fit_transform(data["Time"])
# data["Month"] = data.Date.map(lambda x: int(x.split("/")[1]))
# data["Season"] = (data.Month % 12)//3
# data["Time_cos"] = data.Time.map(np.cos)
# data["Time_sin"] = data.Time.map(np.sin)

# data = pd.get_dummies(data, columns = ["Month", "Season"])
# data = pd.get_dummies(data, columns = ["Season"])
data.drop(columns = ['Date'], inplace = True)


In [4]:
data.head()

,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH
0,18:00:00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578
1,19:00:00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255
2,20:00:00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502
3,21:00:00,2.2,1376.0,80.0,9.2,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867
4,22:00:00,1.6,1272.0,51.0,6.5,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888


In [5]:
len_train_set = int(len(targets) * 0.75)

X_train = data[:len_train_set]
X_test = data[len_train_set:]
y_train = targets[:len_train_set]
y_test = targets[len_train_set:]

## Pipeline

In [6]:
numeric_features = X_train.select_dtypes(include=["int", "float"]).columns.tolist()

numeric_transformer = make_pipeline(StandardScaler())

preprocessor = make_column_transformer((numeric_transformer, numeric_features))

pipeline = make_pipeline(preprocessor, DecisionTreeRegressor(random_state=42))

### SVM model

In [7]:
pipeline = make_pipeline(preprocessor, SVR())
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('svr', SVR())])

In [8]:
param_grid = {
    "svr__kernel": ['rbf'],
    "svr__C": [0.1, 1, 10],
    "svr__degree": [2, 3],  # Only applicable for poly kernel
    "svr__gamma": [0.1, 1, 10],  # Only applicable for poly and rbf kernels
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error', verbose = 2, n_jobs=-1)
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")
print("Best model:", best_pipeline)
print("Best hyperparameters:", best_params)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[CV] END svr__C=0.1, svr__degree=2, svr__gamma=0.1, svr__kernel=rbf; total time=   1.4s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=1, svr__kernel=rbf; total time=   1.4s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=0.1, svr__kernel=rbf; total time=   1.4s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=0.1, svr__kernel=rbf; total time=   1.5s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=0.1, svr__kernel=rbf; total time=   1.5s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=1, svr__kernel=rbf; total time=   1.5s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=1, svr__kernel=rbf; total time=   1.5s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=0.1, svr__kernel=rbf; total time=   1.6s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=10, svr__kernel=rbf; total time=   1.3s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=1, svr__kernel=rbf; total time=   1.4s
[CV] END svr__C=0.1, svr__degree=2, svr__gamma=1, svr__kernel=rbf; total time=   1.5s
[CV] END svr__C=0.1, svr__degree=2, svr__ga

MAE is 4.344159478126345

Best model: Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('svr', SVR(C=10, degree=2, gamma=1))])

Best hyperparameters:
{'svr__C': 10, 'svr__degree': 2, 'svr__gamma': 1, 'svr__kernel': 'rbf'}

### Decision Tree Regressor 

In [9]:
pipeline = make_pipeline(preprocessor, DecisionTreeRegressor(random_state=42))
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

In [10]:
param_grid = {
    "decisiontreeregressor__max_depth": [None, 10, 50],
    "decisiontreeregressor__min_samples_split": [2, 5],
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")
print("Best model:", best_pipeline)
print("Best hyperparameters:", best_params)

MAE is 0.7138461538461539

Best model: Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=42))])

Best hyperparameters:
{'decisiontreeregressor__max_depth': None, 'decisiontreeregressor__min_samples_split': 2}

### Random Forest Regressor

In [11]:
pipeline = make_pipeline(preprocessor, RandomForestRegressor(random_state=42))
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

In [12]:
pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")

MAE is 0.7138461538461539

In [13]:
param_grid = {
    "randomforestregressor__n_estimators": [50, 100],
    "randomforestregressor__max_depth": [None, 10],
    "randomforestregressor__min_samples_split": [2, 5],
    "randomforestregressor__min_samples_leaf": [1, 2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")
print("Best model:", best_pipeline)
print("Best hyperparameters:", best_params)

MAE is 0.5227615384615385

Best model: Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('randomforestregressor',
                 RandomForestRegressor(random_state=42))])

Best hyperparameters:
{
    'randomforestregressor__max_depth': None,
    'randomforestregressor__min_samples_leaf': 1,
    'randomforestregressor__min_samples_split': 2,
    'randomforestregressor__n_estimators': 100
}

## XGBoost

In [14]:
pipeline = make_pipeline(preprocessor, XGBRegressor(random_state=42, n_jobs=-1))
pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              c...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=-1,
                              num_parallel_tree=None, random_state=42, ...))])

In [15]:
pipeline.fit(X_train, y_train)
y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")

MAE is 0.5227615384615385

In [16]:
param_grid = {
    "xgbregressor__n_estimators": [5, 50, 100],
    "xgbregressor__max_depth": [None, 50, 10],
    # "xgbregressor__min_samples_split": [2, 5],
    # "xgbregressor__min_samples_leaf": [1, 2]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train, y_train)

best_pipeline = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_pipeline.predict(X_test)

print(f"MAE is {mean_absolute_error(y_test, y_pred)}")
print("Best model:", best_pipeline)
print("Best hyperparameters:", best_params)

MAE is 0.5337666441334619

Best model: Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  ['CO(GT)', 'PT08.S1(CO)',
                                                   'NMHC(GT)', 'C6H6(GT)',
                                                   'PT08.S2(NMHC)', 'NOx(GT)',
                                                   'PT08.S3(NOx)', 'NO2(GT)',
                                                   'PT08.S4(NO2)',
                                                   'PT08.S5(O3)', 'RH',
                                                   'AH'])])),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              c...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=10, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=100, n_jobs=-1,
                              num_parallel_tree=None, random_state=42, ...))])

Best hyperparameters:
{'xgbregressor__max_depth': 10, 'xgbregressor__n_estimators': 100}